In [1]:
%reload_ext autoreload
%autoreload 2

# Imports

In [2]:
from kret_notebook import *  # NOTE import first
from kret_lgbm._core.lgbm_nb_imports import *
from kret_lightning._core.lightning_nb_imports import *
from kret_matplotlib._core.mpl_nb_imports import *
from kret_np_pd._core.np_pd_nb_imports import *
from kret_optuna._core.optuna_nb_imports import *
from kret_polars._core.polars_nb_imports import *
from kret_rosetta._core.rosetta_nb_imports import *
from kret_sklearn._core.sklearn_nb_imports import *
from kret_torch_utils._core.torch_nb_imports import *
from kret_tqdm._core.tqdm_nb_imports import *
from kret_type_hints._core.types_nb_imports import *
from kret_utils._core.utils_nb_imports import *

# from kret_wandb._core.wandb_nb_imports import *  # NOTE this is slow to import

[kret_np_pd._core.np_pd_nb_imports] Imported kret_np_pd._core.np_pd_nb_imports in 0.0255 seconds
Loaded environment variables from /Users/Akseldkw/coding/kretsinger/.env
[kret_lgbm._core.lgbm_nb_imports] Imported kret_lgbm._core.lgbm_nb_imports in 0.0009 seconds
[kret_lightning._core.lightning_nb_imports] Imported kret_lightning._core.lightning_nb_imports in 0.0413 seconds
[kret_matplotlib._core.mpl_nb_imports] Imported kret_matplotlib._core.mpl_nb_imports in 0.0823 seconds
[kret_optuna._core.optuna_nb_imports] Imported kret_optuna._core.optuna_nb_imports in 0.0002 seconds
[kret_polars._core.polars_nb_imports] Imported kret_polars._core.polars_nb_imports in 0.1227 seconds
[kret_rosetta._core.rosetta_nb_imports] Imported kret_rosetta._core.rosetta_nb_imports in 0.0000 seconds
[kret_sklearn._core.sklearn_nb_imports] Imported kret_sklearn._core.sklearn_nb_imports in 0.1475 seconds
[kret_torch_utils._core.torch_nb_imports] Imported kret_torch_utils._core.torch_nb_imports in 0.5211 seconds


In [3]:
from kret_lightning.examples.cifar10_module import *
from kret_lightning.examples.cifar10_datamodule import *

# Load Data

In [4]:
cifar_dm = CIFAR10DataModule(UKS_CONSTANTS.DATA_DIR / "CIFAR10", batch_size=64)

In [5]:
cifar_dm.data_dir

PosixPath('/Users/Akseldkw/coding/data_kretsinger/CIFAR10')

# Implementation

In [6]:
cifar_nn = CIFAR10ResNet(lr=0.001)
cifar_nn.hparams_initial

Saving hparams, ignoring ()
Saving hparams, ignoring ()


"dropout_rate":     0.5
"l1_penalty":       0.0
"l2_penalty":       0.0
"lr":               0.001
"num_blocks":       3
"num_filters":      64
"number_classes":   10
"patience":         10
"warmup_step_frac": 0.1

In [7]:
cifar_nn.save_load_logging_dict

{'save_dir': PosixPath('/Users/Akseldkw/coding/data_kretsinger/lightning_logs'),
 'name': 'CIFAR10ResNet',
 'version': 'v_000'}

In [8]:
static_args = TrainerStaticDefaults.TRAINER_DEBUG
# static_args["max_epochs"] = 10
static_args

{'detect_anomaly': True, 'fast_dev_run': True}

In [9]:
dynamic_args = TrainerDynamicDefaults.trainer_dynamic_defaults(cifar_nn, cifar_dm)
dynamic_args

{'logger': <lightning.pytorch.loggers.csv_logs.CSVLogger at 0x177ae57c0>,
 'default_root_dir': PosixPath('/Users/Akseldkw/coding/data_kretsinger/lightning_logs/CIFAR10ResNet/v_000'),
 'callbacks': [<lightning.pytorch.callbacks.model_checkpoint.ModelCheckpoint at 0x30475e420>]}

In [ ]:
dynamic_args["logger"].save_dir, dynamic_args["logger"].name

('/Users/Akseldkw/coding/data_kretsinger/lightning_logs', 'CIFAR10ResNet')

In [10]:
trainer_args = static_args | dynamic_args

In [11]:
trainer = L.Trainer(**trainer_args)

You have turned on `Trainer(detect_anomaly=True)`. This will significantly slow down compute speed and is recommended only for model debugging.
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
Running in `fast_dev_run` mode: will run the requested loop using 1 batch(es). Logging and checkpointing is suppressed.


In [ ]:
def objective(trial: optuna.trial.Trial) -> float:
    # We optimize the number of layers, hidden units in each layer and dropouts.
    n_blocks = trial.suggest_int("n_layers", 1, 3)
    dropout = trial.suggest_float("dropout", 0.2, 0.5)
    output_dims = [trial.suggest_int("n_units_l{}".format(i), 4, 128, log=True) for i in range(n_blocks)]

    model = cifar_nn
    datamodule = cifar_dm

    hyperparameters = dict(n_layers=n_blocks, dropout=dropout, output_dims=output_dims)
    assert trainer.logger is not None
    trainer.logger.log_hyperparams(hyperparameters)
    trainer.fit(model, datamodule=datamodule, **TrainerStaticDefaults.TRAINER_FIT)

    return trainer.callback_metrics["val_loss"].item()

In [ ]:
pruner = optuna.pruners.HyperbandPruner()

study = optuna.create_study(
    storage=UKS_CONSTANTS.OPTUNA_STORAGE_DB,
    pruner=pruner,
    study_name="cifar10_resnet",
    direction="minimize",
    load_if_exists=True,
)

[I 2026-01-19 11:04:20,885] Using an existing study with name 'cifar10_resnet' instead of creating a new one.


In [16]:
study.optimize(objective, n_trials=100, timeout=600)

The following callbacks returned in `LightningModule.configure_callbacks` will override existing callbacks passed to Trainer: ModelCheckpoint
100%|██████████| 170M/170M [00:04<00:00, 42.1MB/s] 
/Users/Akseldkw/micromamba/envs/kret_312/lib/python3.12/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:881: Checkpoint directory /Users/Akseldkw/coding/data_kretsinger/lightning_logs/CIFAR10ResNet/v_000 exists and is not empty.
Loading `train_dataloader` to estimate number of stepping batches.


┏━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━┓
┃   ┃ Name           ┃ Type             ┃ Params ┃ Mode  ┃ FLOPs ┃
┡━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━┩
│ 0 │ _train_metrics │ MetricCollection │      0 │ train │     0 │
│ 1 │ _val_metrics   │ MetricCollection │      0 │ train │     0 │
│ 2 │ _test_metrics  │ MetricCollection │      0 │ train │     0 │
│ 3 │ model          │ ResNet           │  4.3 M │ train │     0 │
└───┴────────────────┴──────────────────┴────────┴───────┴───────┘

Trainable params: 4.3 M                                                                                            
Non-trainable params: 0                                                                                            
Total params: 4.3 M                                                                                                
Total estimated model params size (MB): 17                                                                         
Modules in train mode: 82                                                                                          
Modules in eval mode: 0                                                                                            
Total FLOPs: 0

Output()

Metric val_loss improved. New best score: 1.481
Epoch 0, global step 704: 'val_loss' reached 1.48111 (best 1.48111), saving model to '/Users/Akseldkw/coding/data_kretsinger/lightning_logs/CIFAR10ResNet/v_000/best-epoch=00-val_loss=1.48.ckpt' as top 1


...

Trainable params: 4.3 M                                                                                            
Non-trainable params: 0                                                                                            
Total params: 4.3 M                                                                                                
Total estimated model params size (MB): 17                                                                         
Modules in train mode: 82                                                                                          
Modules in eval mode: 0                                                                                            
Total FLOPs: 0

Output()

`Trainer.fit` stopped: `max_epochs=10` reached.


[W 2026-01-19 11:14:55,788] Trial 1 failed with parameters: {'n_layers': 1, 'dropout': 0.24249840967375982, 'n_units_l0': 8} because of the following error: KeyError('val_loss').
Traceback (most recent call last):
  File "/Users/Akseldkw/micromamba/envs/kret_312/lib/python3.12/site-packages/optuna/study/_optimize.py", line 205, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/var/folders/dj/k5kk5wk90vvcnb_wphpw490r0000gn/T/ipykernel_72654/713681628.py", line 14, in objective
    return trainer.callback_metrics["val_loss"].item()
           ~~~~~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^
KeyError: 'val_loss'
[W 2026-01-19 11:14:55,791] Trial 1 failed with value None.


KeyError: 'val_loss'

# Sandbox

In [17]:
trainer.callback_metrics

{}